# Caminhos entre dois vértices

https://networkx.org/documentation/stable/reference/algorithms/simple_paths.html?highlight=path#module-networkx.algorithms.simple_paths

https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html?highlight=path

In [1]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
#import sqlalchemy
import ibm_db
import ibm_db_dbi

In [2]:
#conactando ao BigSql
#colocar credenciais
db = ibm_db.connect("DATABASE=BIGSQL;HOSTNAME=bigsql.pro.intra.rs.gov.br;PORT=32051;PROTOCOL=TCPIP;UID=XXXXXX;PWD=XXXXXXX;", "", "")
conn = ibm_db_dbi.Connection(db)   
#cur = conn.cursor()

In [3]:
# buscando os dados 
query = "select SRC,TGT,REL,WEIGHT from PROCERGS_MILENA_VILLAR.RELACIONAMENTO;"
df = pd.read_sql_query(query, conn)
g = nx.from_pandas_edgelist(df, 'SRC', 'TGT', edge_attr=True)


In [4]:
# criando o grafo não direcionado
G = nx.Graph(g)

In [5]:
#verificando os atributos das arestas
G.edges(1,data=True)

EdgeDataView([(1, 2, {'REL': 'marido', 'WEIGHT': 4}), (1, 3, {'REL': 'mãe', 'WEIGHT': 5}), (1, 4, {'REL': 'mãe', 'WEIGHT': 5}), (1, 16, {'REL': 'irmã', 'WEIGHT': 3}), (1, 14, {'REL': 'mãe', 'WEIGHT': 5}), (1, 15, {'REL': 'pai', 'WEIGHT': 5}), (1, 11, {'REL': 'cunhada', 'WEIGHT': 1}), (1, 10, {'REL': 'irmão', 'WEIGHT': 3}), (1, 5, {'REL': 'sogra', 'WEIGHT': 1})])

In [6]:
# plotar o grafo
nt = Network("400px", "900px", notebook = True)
nt.from_nx(G)
nt.show("grafo.html")

In [7]:
#buscando os labels dos nós(vértices)
query = "select * from PROCERGS_MILENA_VILLAR.PESSOAS;"
df_node = pd.read_sql_query(query, conn)

In [8]:
#transformando a tabela em um dicionário para atribuir labels aos IDs dos nós
nodelabel = df_node.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(G, nodelabel, 'title')
G.nodes(data=True)

NodeDataView({2: {'size': 10, 'title': 'Eloi'}, 1: {'size': 10, 'title': 'Vivianne'}, 3: {'size': 10, 'title': 'Jade'}, 4: {'size': 10, 'title': 'Camila'}, 10: {'size': 10, 'title': 'Fabio'}, 16: {'size': 10, 'title': 'Virginia'}, 11: {'size': 10, 'title': 'Elis'}, 12: {'size': 10, 'title': 'Felipe'}, 13: {'size': 10, 'title': 'Brenda'}, 15: {'size': 10, 'title': 'Ney'}, 14: {'size': 10, 'title': 'Maria'}, 17: {'size': 10, 'title': 'Joaquim'}, 18: {'size': 10, 'title': 'Marisa'}, 7: {'size': 10, 'title': 'Inês'}, 19: {'size': 10, 'title': 'Maiara'}, 20: {'size': 10, 'title': 'Ligia'}, 8: {'size': 10, 'title': 'Barbara'}, 6: {'size': 10, 'title': 'Ricardo'}, 5: {'size': 10, 'title': 'Eduardo'}, 9: {'size': 10, 'title': 'Carolina'}})

In [9]:
#Descobrindo o ID do node Jade
dict(filter(lambda x: x[0] if x[1]['title'] == 'Jade' else False, G.nodes(data=True)))

{3: {'size': 10, 'title': 'Jade'}}

In [10]:
#Descobrindo o ID do node Marisa
dict(filter(lambda x: x[0] if x[1]['title'] == 'Marisa' else False, G.nodes(data=True)))

{18: {'size': 10, 'title': 'Marisa'}}

In [11]:
# Jade e Marisa são amigas?
print(G.has_edge(3,18))

False


In [12]:
#outra opção
def nodes_connected(u, v):
    return u in G.neighbors(v)

nodes_connected(3,18)

False

In [13]:
#Existe um caminho entre os vértices?
nx.has_path(G,3,18)

True

## Algoritmo Dijkstra
Soluciona o problema do caminho mais curto em grafos com arestas de peso não negativo.
O algoritmo que serve para resolver o mesmo problema em um grafo com pesos negativos é o algoritmo de Bellman-Ford,
que possui maior tempo de execução que o Dijkstra.

https://www.youtube.com/watch?v=ovkITlgyJ2s

https://www.youtube.com/watch?v=aJ_2c9NVCIc

In [14]:
# Descobre o caminho de custo mínimo (ccm) do vértice Jade até Marisa // o peso das arestas é considerado
ccm = nx.dijkstra_path(G,source = 3, target = 18, weight = 'WEIGHT') 
tam = nx.dijkstra_path_length(G,source = 3, target = 18, weight = 'WEIGHT')
print('Caminho de custo mínimo: ', ccm)
print('Tamanho do caminho: ', tam) # soma dos pesos das arestas

Caminho de custo mínimo:  [3, 5, 6, 18]
Tamanho do caminho:  3


In [15]:
#outra forma
#O caminho mais curto
ccm = nx.shortest_path(G, 3, 18, weight = 'WEIGHT', method= 'dijkstra') # method= 'bellman-ford' (para pesos/valores negativos), 'dijkstra'
# tamanho do caminho
tam =nx.shortest_path_length(G, 3, 18, weight = 'WEIGHT', method= 'dijkstra') # method= 'bellman-ford', 'dijkstra'
print('Caminho de custo mínimo: ', ccm)
print('Tamanho do caminho: ', tam)

Caminho de custo mínimo:  [3, 5, 6, 18]
Tamanho do caminho:  3


In [16]:
# Gerando todos os caminhos de Jade até Marisa.
#Um caminho simples é um caminho sem nós repetidos
for path in nx.all_simple_paths(G, source=3, target=18, cutoff=3): # cutoff é o tamanho do caminho
    print(path)


[3, 4, 8, 18]
[3, 4, 9, 18]
[3, 4, 6, 18]
[3, 4, 7, 18]
[3, 5, 8, 18]
[3, 5, 6, 18]
[3, 5, 9, 18]


In [17]:
# Mostra todos as arestas que ligam cada vértice do caminho
for path in nx.all_simple_edge_paths(G, 3, 18, cutoff = 3):
    print(path)

[(3, 4), (4, 8), (8, 18)]
[(3, 4), (4, 9), (9, 18)]
[(3, 4), (4, 6), (6, 18)]
[(3, 4), (4, 7), (7, 18)]
[(3, 5), (5, 8), (8, 18)]
[(3, 5), (5, 6), (6, 18)]
[(3, 5), (5, 9), (9, 18)]


In [18]:
d =  G.get_edge_data(3,4)['WEIGHT'] + G.get_edge_data(4,8)['WEIGHT'] + G.get_edge_data(8,18)['WEIGHT']
d

10

In [19]:
d =  G.get_edge_data(3,5)['WEIGHT'] + G.get_edge_data(5,6)['WEIGHT'] + G.get_edge_data(6,18)['WEIGHT']
d

3

In [21]:
# outro exemplo
# Descobre o caminho de custo mínimo (ccm) do vértice // o peso das arestas é considerado
ccm = nx.dijkstra_path(G,source = 1, target =20, weight = 'WEIGHT') 
tam = nx.dijkstra_path_length(G,source = 1, target = 20, weight = 'WEIGHT')
print('Caminho de custo mínimo: ', ccm)
print('Tamanho do caminho: ', tam)

Caminho de custo mínimo:  [1, 5, 3, 13, 4, 20]
Tamanho do caminho:  7
